## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 셀레니움 사용 자동화

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈
import re   # 정규식 처리 모듈
import time
from PIL import Image,ImageOps
from tqdm import tqdm
from collections import Counter
from wordcloud import WordCloud
import platform
import folium
from folium.plugins import MarkerCluster # 마커클러스터용 클래스 추가
import requests # 인터넷에서 받아오는 라이브러리

In [2]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font)
rcParams['axes.unicode_minus'] = False

In [3]:
#불필요한 경고메시지 제거
import warnings
warnings.filterwarnings('ignore')   

In [10]:
# 웹드라이버 스타벅스 웹사이트 오픈
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [14]:
# 서울링크 클릭
path1 = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, path1).click()

In [ ]:
# 서울클릭 다른 방법
''' 
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()
'''

In [16]:
# 웹드라이버에서 소스 위치, 전체링크 클릭
name = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

##### BeautifulSoup로 html 가져오기

In [45]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [46]:
sbseoullist = soup.select('li.quickResultLstCon')

In [ ]:
sbseoullist

##### 서울 매장 리스트 DF

In [59]:
sbseoulstores = []
for i in sbseoullist:
    name = i.select('strong')[0].text.strip()   # 가게명
    lat = i['data-lat'].strip()     # 위도
    lng = i['data-long'].strip()    # 경도
    storeType = i.select('i')[0]['class'][0].split('_')[1] # 가게종류(일반, 리저브, DT, WT)
    address = str(i.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(i.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]  # 전화번호(1522-3232 대표번호로 변경)
    sbseoulstores.append([name,lat,lng,storeType,address,tel])


In [60]:
dfStarBucksSeoul = pd.DataFrame(sbseoulstores, columns=['매장명','위도','경도','매장타입','주소','전화번호'])

In [61]:
dfStarBucksSeoul

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [62]:
dfStarBucksSeoul.to_excel('./data/starbucks_seoul_list.xlsx')

In [64]:
### 서울열린데이터광장 OpenAPI

key = '5751755353746c613535496d704867'

In [65]:
url = f'http://openapi.seoul.go.kr:8088/{key}/json/SdeTlSccoSigW/1/25'
url

'http://openapi.seoul.go.kr:8088/5751755353746c613535496d704867/json/SdeTlSccoSigW/1/25'

In [ ]:
# 서울시 지도
https://github.com/southkorea/seoul-maps/blob/master/juso/2015/json/seoul_municipalities_geo_simple.json